In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ingredient substitution'

Mounted at /content/gdrive
/content/gdrive/MyDrive/ingredient substitution/Exploiting-Food-Embeddings-for-Ingredient-Substitution


In [ ]:
#Load custom ingredients lists and custom vocab
import json
with open('./Food_BERT_model/used_ingredients.json', 'r') as f:
    used_ingredients = json.load(f)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.6 MB/s 
     |████████████████████████████████| 6.6 MB 38.4 MB/s 
     |████████████████████████████████| 596 kB 44.7 MB/s 
     |████████████████████████████████| 101 kB 9.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased",output_hidden_states = True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# empty list to read list from a file
final_datalist = []

# open file and read the content in a list
with open(r'./data/1100kinstruction.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        final_datalist.append(x)

In [ ]:
len(final_datalist)

100000

In [ ]:
final_datalist[0]

' Preheat the oven to 350 F. Butter or oil an 8-inch baking dish. Cook the penne 2 minutes less than package directions. (It will finish cooking in the oven.) Rinse the pasta in cold water and set aside. Combine the cooked pasta and the sauce in a medium bowl and mix carefully but thoroughly. Scrape the pasta into the prepared baking dish. Sprinkle the top with the cheeses and then the chili powder. Bake, uncovered, for 20 minutes. Let the mac and cheese sit for 5 minutes before serving. Melt the butter in a heavy-bottomed saucepan over medium heat and whisk in the flour. Continue whisking and cooking for 2 minutes. Slowly add the milk, whisking constantly. Cook until the sauce thickens, about 10 minutes, stirring frequently. Remove from the heat. Add the cheeses, salt, chili powder, and garlic powder. Stir until the cheese is melted and all ingredients are incorporated, about 3 minutes. Use immediately, or refrigerate for up to 3 days. This sauce reheats nicely on the stove in a sauce

In [ ]:
used_ingredients_without_underscore = []
import re
for ing in used_ingredients:
  result = re.sub('_', ' ', ing)  
  used_ingredients_without_underscore.append(result)

In [ ]:
#replace every ingredient with its underscore version
new_datalist = []
for data in final_datalist:
  for i in range(len(used_ingredients_without_underscore)):
    if(used_ingredients_without_underscore[i] in data):
      data = re.sub(used_ingredients_without_underscore[i],used_ingredients[i],data)
  new_datalist.append(data)

In [ ]:
new_datalist[0]

' Preheat the oven to 350 F. Butter or oil an 8-inch baking dish. Cook the penne 2 minutes less than package directions. (It will finish cooking in the oven.) Rinse the pasta in cold_water and set aside. Combine the cooked pasta and the sauce in a medium bowl and mix carefully but thoroughly. Scrape the pasta into the prepared baking dish. Sprinkle the top with the cheeses and then the chili_powder. Bake, uncovered, for 20 minutes. Let the mac and cheese sit for 5 minutes before serving. Melt the butter in a heavy-bottomed saucepan over medium heat and whisk in the flour. Continue whisking and cooking for 2 minutes. Slowly add the milk, whisking constantly. Cook until the sauce thickens, about 10 minutes, stirring frequently. Remove from the heat. Add the cheeses, salt, chili_powder, and garlic_powder. Stir until the cheese is melted and all ingredients are incorporated, about 3 minutes. Use immediately, or refrigerate for up to 3 days. This sauce reheats nicely on the stove in a sauce

In [ ]:
import nltk
from nltk import sent_tokenize,word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
test = new_datalist[:5000]

In [ ]:
embedding_dt = {}

In [ ]:
#last layer
from operator import add
k = 1
for data in test:
  if(k % 250 == 0):
    print(k)
  k = k+1
  sent_list = sent_tokenize(data)
  for sent in sent_list:
    sent = sent.lower()
    encoded_input = tokenizer(sent, return_tensors='tf')
    outputs = model(encoded_input)
    all_token_emb = outputs[0][0].numpy().tolist()
    word_list = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0].numpy().tolist())
    for i in range(len(word_list)):
      if(word_list[i] in used_ingredients):
        if word_list[i] in embedding_dt:
          embedding_dt[word_list[i]] = [x / 2 for x in map(add, embedding_dt[word_list[i]],all_token_emb[i])]
        else:
          embedding_dt[word_list[i]] = all_token_emb[i]

In [ ]:
len(embedding_dt)

250

In [ ]:
#saving
import pickle
with open('./Base_Bert_Embeddings/embedding_last_layer_5K.pkl', 'wb') as f:
    pickle.dump(embedding_dt, f)

After 5k recipes only 250 known ingredings are found. This is because all the ingredients not present in bert vocab are marked with same token i.e. token 100. So all are considered same. With such small number of ingredients performance is bound to be bad and investigating for mean and weighted average is not needed.